# Building Agents 
 
> Note: Optionally, see [these slides](https://docs.google.com/presentation/d/13c0L1CQWAL7fuCXakOqjkvoodfynPJI4Hw_4H76okVU/edit?usp=sharing) and [langgraph_101.ipynb](langgraph_101.ipynb) for context before diving into this notebook!

We're going to build an email assistant from scratch, starting here with 1) the agent architecture (using [LangGraph JS](https://langchain-ai.github.io/langgraphjs/)) and following with 2) testing (using [LangSmith](https://docs.smith.langchain.com/)), 3) human-in-the-loop, and 4) memory. This diagram show how these pieces will fit together:

![overview-img](img/overview.png)

### Tool Definition

Let's start by defining some simple tools that an email assistant will use with the `@tool` decorator:

In [5]:
import { z } from "zod";
import { tool, DynamicStructuredTool } from "@langchain/core/tools";

// Define schema for email writing
const writeEmailSchema = z.object({
  to: z.string().describe("Email address of the recipient"),
  subject: z.string().describe("Subject line for the email"),
  content: z.string().describe("Main body text of the email")
});

// Write email tool
const writeEmail = tool(async (input: z.infer<typeof writeEmailSchema>) => {
  const { to, subject, content } = input;
  // Placeholder response - in real app would send email
  return `Email sent to ${to} with subject '${subject}' and content: ${content}`;
}, {
  name: "write_email",
  description: "Write and send an email.",
  schema: writeEmailSchema,
});

// Define schema for scheduling meeting
const scheduleMeetingSchema = z.object({
  attendees: z.array(z.string()).describe("List of attendees' emails"),
  subject: z.string().describe("Meeting title or subject"),
  duration_minutes: z.number().describe("Duration of meeting in minutes"),
  preferred_day: z.string().describe("Preferred date for the meeting (e.g., 'tomorrow', '2024-07-28')"),
  start_time: z.string().describe("Start time of the meeting (e.g., '2:00 PM', '14:00')")
});

// Schedule meeting tool
const scheduleMeeting = new DynamicStructuredTool({
  name: "schedule_meeting",
  description: "Schedule a calendar meeting.",
  schema: scheduleMeetingSchema,
  func: async (input: z.infer<typeof scheduleMeetingSchema>) => {
    const { attendees, subject, duration_minutes, preferred_day, start_time } = input;
    // Placeholder response - in real app would check calendar and schedule
    // Basic date parsing, consider a robust library for production
    let dateStr = preferred_day;
    try {
        // Attempt to format if it's a parsable date string
        const parsedDate = new Date(preferred_day);
        if (!isNaN(parsedDate.getTime())) {
             dateStr = parsedDate.toLocaleDateString('en-US', { weekday: 'long', month: 'long', day: 'numeric', year: 'numeric' });
        }
        // If preferred_day is not a valid date string (e.g. "tomorrow"), use it as is.
    } catch (e) {
        // Fallback to using preferred_day as is if parsing fails
    }
    return `Meeting '${subject}' scheduled on ${dateStr} at ${start_time} for ${duration_minutes} minutes with ${attendees.join(', ')}`;
  }
});

// Define schema for checking calendar
const checkCalendarSchema = z.object({
  day: z.string().describe("Day to check calendar availability (e.g., 'tomorrow', 'next Monday', '2024-07-28')")
});

// Check calendar availability tool
const checkCalendarAvailability = new DynamicStructuredTool({
  name: "check_calendar_availability",
  description: "Check calendar availability for a given day.",
  schema: checkCalendarSchema,
  func: async (input: z.infer<typeof checkCalendarSchema>) => {
    const { day } = input;
    // Placeholder response - in real app would check actual calendar
    return `Available times on ${day}: 9:00 AM, 2:00 PM, 4:00 PM`;
  }
});

// Define schema for Done tool
const doneSchema = z.object({
  // The LLM will decide if the task is 'done' and pass true/false.
  done: z.boolean().describe("Task completion status. Set to true when the current task or series of actions is complete.")
});

// TODO FIX TOOL (Original notebook comment: Consider if the 'done' boolean input is truly necessary from the LLM,
// or if simply invoking this tool implies completion, in which case the schema could be z.object({}))
// Done tool
const Done = new DynamicStructuredTool({
  name: "Done",
  description: "Signals that the current e-mail related task or sequence of tool calls is complete. Call this when no more actions are needed for the current e-mail.",
  schema: doneSchema,
  func: async (input: z.infer<typeof doneSchema>) => {
    // The observation returned to the LLM.
    // The 'input.done' (true/false from LLM) is used here to make the observation more specific.
    return input.done ? "Task marked as completed successfully by LLM." : "Task completion not explicitly confirmed as true by LLM, but Done tool was called.";
  }
});

### Augmenting the LLM with Tools

Now we connect these tools to a LLM. We'll use LangChain's [`initChatModel`](https://js.langchain.com/docs/how_to/chat_models_universal_init/) interface, which allows us to initialize many different chat models.

We [enforce tool use](https://js.langchain.com/docs/how_to/tool_choice/) by setting `tool_choice: "required"` (or the equivalent parameter for the specific model integration, like in `bindTools`).

In [19]:
// Cell 2 without IIFE
import { initChatModel } from "langchain/chat_models/universal";
import { AIMessage, BaseMessage } from "@langchain/core/messages";

// Declare 'output' in a scope accessible by subsequent cells
let output: AIMessage | null = null;

// --- IIFE removed, code now at top level of cell ---
try {
  const llm = await initChatModel(
    "openai:gpt-4o",
    { temperature: 0.0 }
  );

  const llmWithTools = llm.bindTools([writeEmail]); // Assumes writeEmail is defined

  const emailInput = {
    author: "System Admin <sysadmin@company.com>",
    to: "Development Team <dev@company.com>",
    subject: "Scheduled maintenance - database downtime",
    email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
  };

  const messages: BaseMessage[] = [
      {
      role: "user",
      content: "Call the 'write_email' tool in order to respond to this email: " + JSON.stringify(emailInput, null, 2)
      } as any
  ];

  output = await llmWithTools.invoke(messages);

  console.log("LLM Output from Cell 2:", output);
  if (output && output.tool_calls) {
    console.log("Tool Calls from Cell 2:", output.tool_calls);
  } else {
    console.log("No tool calls found in the output from Cell 2.");
  }

} catch (error) {
  console.error("Error in cell 2 execution:", error);
  output = null;
}
// --- End of cell code ---

LLM Output from Cell 2: AIMessage {
  "id": "chatcmpl-BVqqUq7M43a1SlEWtexkTIXliC2xF",
  "content": "",
  "additional_kwargs": {
    "tool_calls": [
      {
        "id": "call_98EW3IPCPkSEuVCmXRIHvtRU",
        "type": "function",
        "function": "[Object]"
      }
    ]
  },
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 200,
      "completionTokens": 73,
      "totalTokens": 273
    },
    "finish_reason": "tool_calls",
    "model_name": "gpt-4o-2024-08-06",
    "usage": {
      "prompt_tokens": 200,
      "completion_tokens": 73,
      "total_tokens": 273,
      "prompt_tokens_details": {
        "cached_tokens": 0,
        "audio_tokens": 0
      },
      "completion_tokens_details": {
        "reasoning_tokens": 0,
        "audio_tokens": 0,
        "accepted_prediction_tokens": 0,
        "rejected_prediction_tokens": 0
      }
    },
    "system_fingerprint": "fp_f5bdcc3276"
  },
  "tool_calls": [
    {
      "name": "write_email",
      "args": {
       

In [20]:
console.log(output.tool_calls);

[
  {
    name: 'write_email',
    args: {
      to: 'sysadmin@company.com',
      subject: 'Re: Scheduled maintenance - database downtime',
      content: 'Hi System Admin Team,\n' +
        '\n' +
        'Thank you for the reminder. We will ensure that no critical deployments are scheduled during the maintenance window and will plan our work accordingly.\n' +
        '\n' +
        'Best regards,\n' +
        'Development Team'
    },
    type: 'tool_call',
    id: 'call_98EW3IPCPkSEuVCmXRIHvtRU'
  }
]


In [25]:
const args = output.tool_calls?.[0]?.args;


In [26]:

const writeEmailResult = await writeEmail.invoke(args);
console.log(writeEmailResult);

Email sent to sysadmin@company.com with subject 'Re: Scheduled maintenance - database downtime' and content: Hi System Admin Team,

Thank you for the reminder. We will ensure that no critical deployments are scheduled during the maintenance window and will plan our work accordingly.

Best regards,
Development Team


## Building our email assistant

We'll combine a [router and agent](https://langchain-ai.github.io/langgraphjs/tutorials/workflows/) to build our email assistant.

![agent_workflow_img](img/email_workflow.png)

### Router

The routing step handles the triage decision. 

The triage router only focuses on the triage decision, while the agent focuses *only* on the response. 

When building an agent, it's important to consider the information that you want to track over time. LangGraph uses a state object to manage this. For chat history, a common approach is to use a `messages` key in the state, often managed by a reducer like the one provided by `MessagesAnnotation` (which internally uses `messagesStateReducer`). This reducer appends new messages (instances of `BaseMessage` or its subclasses like `AIMessage`, `HumanMessage`) to the history and can also handle `RemoveMessage` objects to delete specific messages. This allows for all standard message types to be part of the state. However, LangGraph gives you flexibility to track other information beyond messages. We'll define a custom `State` object (typically using Zod for schema definition in TypeScript) that includes a `messages` key for our chat history (allowing for `BaseMessage` types) and adds a `classification_decision` key:

In [27]:
import { z } from "zod";

// to do replace with Proper State for Messages 
// Create StateSchema using Zod
const StateSchema = 
z.object({
  messages: z.array(z.any()),
  email_input: z.record(z.any()),
  classification_decision: z.enum(["ignore", "respond", "notify"])
});




#### Triage node

We define a TypeScript function with our triage routing logic.

For this, we use [structured outputs](https://js.langchain.com/docs/concepts/structured_outputs/) with a Zod schema. Zod schemas are particularly useful for defining structured output schemas in TypeScript because they offer type hints and validation. The descriptions in the Zod schema are important because they get passed as part of the JSON schema to the LLM to inform the output coercion (often using the `.withStructuredOutput()` method).

In [43]:
import { END, Command } from "@langchain/langgraph";
import { initChatModel } from "langchain/chat_models/universal";
import { z } from "zod";
import { BaseMessage, HumanMessage } from "@langchain/core/messages";

// Define the state schema (matching what would be in schemas.js)
const StateSchema = z.object({
  messages: z.array(z.any()).describe("The history of messages in the conversation"),
  email_input: z.record(z.any()).describe("The raw input email data"),
  classification_decision: z.enum(["ignore", "respond", "notify"]).optional()
    .describe("The classification decision made by the triage router")
});

// Define the state type from the schema
type State = z.infer<typeof StateSchema>;

// Define prompt templates directly in this cell rather than importing
const triageSystemPrompt = `As an email assistant, analyze each email carefully. 
{background} 
{triage_instructions}`;

const triageUserPrompt = `From: {author}
To: {to}
Subject: {subject}

Email Content:
{email_thread}`;

const defaultBackground = `You work as an email assistant for a busy professional who receives many emails.
Your job is to analyze incoming emails and determine which ones need attention.`;

const defaultTriageInstructions = `Classify each email into one of these categories:
- "ignore" - Spam, marketing, or messages not requiring any action
- "respond" - Emails that need a direct response
- "notify" - Important information that should be noted but doesn't require a response`;


// Tool descriptions for agent workflow without triage
 const AGENT_TOOLS_PROMPT = `
1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Done - E-mail has been sent
`;

// Utility functions defined inline
const parseEmail = (email_input: Record<string, any>) => {
  if (!email_input) {
    throw new Error("Email input is missing or undefined");
  }
  
  return {
    author: email_input.author || "",
    to: email_input.to || "",
    subject: email_input.subject || "",
    // Handle both email_thread and emailThread fields for flexibility
    emailThread: email_input.email_thread || email_input.emailThread || ""
  };
};

const formatEmailMarkdown = (subject: string, author: string, to: string, emailThread: string) => {
  return `## Email Details
**Subject:** ${subject}
**From:** ${author}
**To:** ${to}

### Content:
${emailThread}`;
};

// Triage router node function
async function triageRouter(state: State) {
  try {
    const { email_input } = state;
    if (!email_input) {
      throw new Error("email_input is missing from state");
    }
    
    // Parse the email
    const parseResult = parseEmail(email_input);
    
    // Extract the needed components
    const { author, to, subject, emailThread } = parseResult;
    
    // Format the prompts with the email details
    const systemPrompt = triageSystemPrompt
      .replace("{background}", defaultBackground)
      .replace("{triage_instructions}", defaultTriageInstructions);
      
    const userPrompt = triageUserPrompt
      .replace("{author}", author)
      .replace("{to}", to)
      .replace("{subject}", subject)
      .replace("{email_thread}", emailThread);
      
    // Create email markdown for potential response  
    const emailMarkdown = formatEmailMarkdown(
      subject,
      author,
      to,
      emailThread
    );
    
    // Add clear instruction for simple response
    const simplifiedSystemPrompt = `${systemPrompt}

After analyzing this email, determine if it should be:
1. "ignore" - Not important, no action needed
2. "respond" - Requires a response
3. "notify" - Contains important information but no response needed

Reply with ONLY ONE WORD: "ignore", "respond", or "notify".`;
    
    // Initialize the basic LLM
    const llm = await initChatModel("openai:gpt-4", { temperature: 0.0 });
    
    // Invoke the LLM with the simplified prompt
    const response = await llm.invoke([
      { role: "system", content: simplifiedSystemPrompt },
      { role: "human", content: userPrompt }
    ]);
    
    // Extract classification from simple response text
    let classification: "ignore" | "respond" | "notify" | undefined;
    const responseText = (response.content || "")
      .toString()
      .toLowerCase()
      .trim();
      
    if (responseText.includes("respond")) {
      classification = "respond";
    } else if (responseText.includes("notify")) {
      classification = "notify";
    } else if (responseText.includes("ignore")) {
      classification = "ignore";
    } else {
      console.log(`Unrecognized classification: "${responseText}". Defaulting to notify.`);
      classification = "notify";
    }
    
    // Determine next steps based on classification
    let goto = END;
    let update: Partial<State> = {
      classification_decision: classification,
    };
    
    if (classification === "respond") {
      console.log("📧 Classification: RESPOND - This email requires a response");
      goto = "response_agent";
      
      update.messages = [
        new HumanMessage({
          content: `Respond to the email: ${emailMarkdown}`
        })
      ];
    } else if (classification === "ignore") {
      console.log("🚫 Classification: IGNORE - This email can be safely ignored");
    } else if (classification === "notify") {
      console.log("🔔 Classification: NOTIFY - This email contains important information");
    } else {
      throw new Error(`Invalid classification: ${classification}`);
    }
    
    // Return command with routing and state update
    return new Command({
      goto,
      update
    });
  } catch (error) {
    console.error("Error in triage router:", error);
    // Default to END in case of errors
    return new Command({
      goto: END,
      update: {
        classification_decision: "ignore",
        messages: [
          {
            role: "system",
            content: `Error processing email: ${error instanceof Error ? error.message : String(error)}`
          }
        ]
      }
    });
  }
}



We use concepts similar to [Commands (how to update state and jump to nodes)](https://langchain-ai.github.io/langgraphjs/how-tos/command/) in LangGraph to both update the state and select the next node to visit. This is a useful alternative to edges.

### Agent

Now, let's build the agent.

#### LLM node

Here, we define the LLM decision-making node. This node takes in the current state, calls the LLM, and updates `messages` with the LLM output. 

In [44]:
console.log(AGENT_TOOLS_PROMPT);


1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day, start_time) - Schedule calendar meetings where preferred_day is a datetime object
3. check_calendar_availability(day) - Check available time slots for a given day
4. Done - E-mail has been sent



In [45]:
console.log(agentSystemPrompt);

1:13 - Cannot find name 'agentSystemPrompt'. Did you mean 'triageSystemPrompt'?


In [63]:

// Collect all tools
const tools = [writeEmail, scheduleMeeting, checkCalendarAvailability, Done];
const toolsByName = new Map(tools.map(tool => [tool.name, tool]));
import llmCallNode from "./src/email_assistant";

// Initialize the LLM, enforcing tool use
 // Initialize the LLM
 const llm = await initChatModel("openai:gpt-4", {
  temperature: 0.0,
});


const llmWithTools = llm.bindTools(tools, { toolChoice: "required" });
const agentSystemPrompt = `
<Role>
You are a top-notch executive assistant who cares about helping your executive perform as well as possible.
</Role>

<Tools>
You have access to the following tools to help manage communications and schedule:
{tools_prompt}
</Tools>

<Instructions>
When handling emails, follow these steps:
1. Carefully analyze the email content and purpose
2. IMPORTANT --- always call a tool and call one tool at a time until the task is complete: 
3. For responding to the email, draft a response email with the write_email tool
4. For meeting requests, use the check_calendar_availability tool to find open time slots
5. To schedule a meeting, use the schedule_meeting tool with a datetime object for the preferred_day parameter
   - Today's date is ${new Date().toISOString().split("T")[0]} - use this for scheduling meetings accurately
6. If you scheduled a meeting, then draft a short response email using the write_email tool
7. After using the write_email tool, the task is complete
8. If you have sent the email, then use the Done tool to indicate that the task is complete
</Instructions>

<Background>
{background}
</Background>

<Response Preferences>
{response_preferences}
</Response Preferences>

<Calendar Preferences>
{cal_preferences}
</Calendar Preferences>
`;
  // Create the LLM call node
  const llmCallNode = async (state) => {
    /**
     * LLM decides whether to call a tool or not
     * This is the main decision-making node that generates responses or tool calls
     */
    const messages = [...state.messages];
    const systemPromptContent = agentSystemPrompt
      .replace("{tools_prompt}", AGENT_TOOLS_PROMPT)
      .replace("{background}", defaultBackground)
      // .replace("{response_preferences}", defaultResponsePreferences)
      // .replace("{cal_preferences}", defaultCalPreferences);

    // Run the LLM with the messages
    const response = await llmWithTools.invoke([
      { role: "system", content: systemPromptContent },
      ...messages,
    ]);

    // Use explicit casting as the response is compatible with Message in runtime
    return {
      messages: response,
    };
  };


5:8 - Module '"/Users/dylan/Desktop/agents-from-scratch-ts/notebooks/src/email_assistant"' has no default export.


#### Tool handler node

After the LLM makes a decision, we need to execute the chosen tool. The `toolHandler` node executes the tool. We can see that nodes can update the graph state to capture any important state changes, such as the classification decision.

In [64]:
// Tool handler function
async function toolHandler() {
  // List for tool messages
  const result = [];
  
  // Get the last message
  const lastMessage = state.messages[state.messages.length - 1];
  
  // Iterate through tool calls
  for (const toolCall of lastMessage.tool_calls || []) {
    // Get the tool
    const tool = toolsByName.get(toolCall.name);
    
    if (!tool) {
      throw new Error(`Tool ${toolCall.name} not found`);
    }
    
    // Run it
    const observation = await tool.invoke(toolCall.args);
    
    // Create a tool message
    result.push({
      role: "tool",
      content: observation,
      tool_call_id: toolCall.id
    });
  }
  
  // Add it to our messages
  return { messages: result };
}

7:23 - Cannot find name 'state'.
7:38 - Cannot find name 'state'.
12:18 - Cannot find name 'toolsByName'.


#### Conditional Routing

Our agent needs to decide when to continue using tools and when to stop. This conditional routing function directs the agent to either continue or terminate.

In [65]:
// Function to determine if we should continue or end
function shouldContinue() {
  // Get the last message
  const messages = state.messages;
  const lastMessage = messages[messages.length - 1];
  
  // If the last message is a tool call, check if it's a Done tool call
  if (lastMessage.tool_calls && lastMessage.tool_calls.length > 0) {
    for (const toolCall of lastMessage.tool_calls) {
      if (toolCall.name === "Done") {
        return END;
      }
    }
    return "tool_handler";
  }
  
  // Default case - shouldn't reach here
  return "tool_handler";
}

4:20 - Cannot find name 'state'.


#### Agent Graph

Finally, we can assemble all components:

In [66]:
import { StateGraph, START, END } from "@langchain/langgraph";

// Build workflow
const overallWorkflow = new StateGraph<State>({
  channels: StateSchema
})
  .addNode("llm_call", llmCall)
  .addNode("tool_handler", toolHandler);

// Add edges
overallWorkflow.addEdge(START, "llm_call");
overallWorkflow.addConditionalEdges(
  "llm_call",
  shouldContinue,
  {
    "tool_handler": "tool_handler",
    [END]: END,
  }
);
overallWorkflow.addEdge("tool_handler", "llm_call");

// Compile the agent
const agent = overallWorkflow.compile();

// In a TypeScript environment, we would visualize it with:
// agent.showGraph();




4:25 - No overload matches this call.
4:25 - Overload 1 of 3, '(fields: never, configSchema?: StateDefinition | AnnotationRoot<StateDefinition>): StateGraph<{ messages?: any[]; email_input?: Record<string, any>; classification_decision?: "ignore" | ... 1 more ... | "notify"; }, ... 5 more ..., StateDefinition>', gave the following error.
4:25 - Argument of type '{ channels: z.ZodObject<{ messages: z.ZodArray<z.ZodAny, "many">; email_input: z.ZodRecord<z.ZodString, z.ZodAny>; classification_decision: z.ZodOptional<z.ZodEnum<["ignore", "respond", "notify"]>>; }, "strip", z.ZodTypeAny, { ...; }, { ...; }>; }' is not assignable to parameter of type 'never'.
4:25 - Overload 2 of 3, '(fields: StateGraphArgs<{ messages?: any[]; email_input?: Record<string, any>; classification_decision?: "ignore" | "respond" | "notify"; }>, configSchema?: StateDefinition | AnnotationRoot<...>): StateGraph<...>', gave the following error.
4:25 - Type 'ZodObject<{ messages: ZodArray<ZodAny, "many">; email_input

This creates a graph that:
1. Starts with an LLM decision
2. Conditionally routes to tool execution or termination
3. After tool execution, returns to LLM for the next decision
4. Repeats until completion or no tool is called


### Combine workflow with our agent

We can combine the router and the agent.

In [67]:
// Combine the triage router and agent workflow
const combinedWorkflow = new StateGraph<State>({
  channels: StateSchema
})
  .addNode("triage_router", triageRouter)
  .addNode("response_agent", agent)
  .addEdge(START, "triage_router");

// Compile the combined workflow
const finalWorkflow = combinedWorkflow.compile();

 finalWorkflow.showGraph();

2:30 - Cannot find name 'StateGraph'.
6:30 - Cannot find name 'agent'.
7:12 - Cannot find name 'START'.


This is a higher-level composition where:
1. First, the triage router analyzes the email
2. If needed, the response agent handles crafting a response
3. The workflow ends when either the triage decides no response is needed or the response agent completes

In [68]:
// Test email input
const testEmailInput = {
  author: "System Admin <sysadmin@company.com>",
  to: "Development Team <dev@company.com>",
  subject: "Scheduled maintenance - database downtime",
  email_thread: "Hi team,\n\nThis is a reminder that we'll be performing scheduled maintenance on the production database tonight from 2AM to 4AM EST. During this time, all database services will be unavailable.\n\nPlease plan your work accordingly and ensure no critical deployments are scheduled during this window.\n\nThanks,\nSystem Admin Team"
};

// Run the agent
const response = await finalWorkflow.invoke({
  email_input: testEmailInput,
  messages: [],
  classification_decision: null  // Will be set during execution
});

// In a real TypeScript environment, we would display messages like:
// response.messages.forEach(m => console.log(m));


10:24 - Cannot find name 'finalWorkflow'.


In [69]:
// Second test email input
const testEmailInput2 = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

// Run the agent with the second test input
const response2 = await finalWorkflow.invoke({
  email_input: testEmailInput2,
  messages: [],
  classification_decision: null  // Will be set during execution
});

// In a real TypeScript environment, we would display messages like:
// response2.messages.forEach(m => console.log(m));


10:25 - Cannot find name 'finalWorkflow'.


## Testing with Local Deployment

You can find the file for our agent in the `src` directory:

* `src/email_assistant.ts`

You can test them locally in LangGraph Studio by running:

```bash
pnpm agent
```

Example e-mail you can test:

In [70]:
const emailInputExample = {
  author: "Alice Smith <alice.smith@company.com>",
  to: "John Doe <john.doe@company.com>",
  subject: "Quick question about API documentation",
  email_thread: "Hi John,\nI was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?\nSpecifically, I'm looking at:\n- /auth/refresh\n- /auth/validate\nThanks!\nAlice"
};

![studio-img](img/studio.png)